Let's make sure we are using tensorflow v2+

# Training and inference with colab
[Google colab](https://colab.research.google.com) is a free server for running notebooks with GPU/TPU support - this is a great way to use _DAS_ if you do not have a computer with a GPU.

This notebook demonstrates 
- how to setup _DAS_ 
- how to load your own datasets
- how to train a network and then use that network to label a new audio recording.

Open and edit this notebook in colab by clicking this badge:

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/janclemenslab/das/blob/master/colab/colab.ipynb)

Install _DAS_:

In [ ]:
!pip install das "tensorflow<2.16"

Import all the things:

In [ ]:
import das.train, das.predict, das.utils, das.npy_dir
import matplotlib.pyplot as plt
import flammkuchen
import logging
logging.basicConfig(level=logging.INFO)

Mount your google drive so you can access your own datasets - this will ask for authentication.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Train the model
Adjust the variable `path_to_data` to point to the dataset on your own google drive.

In [ ]:
path_to_data = '/content/drive/MyDrive/Dmoj.wrigleyi.npy'

das.train.train(model_name='tcn',
                data_dir=path_to_data,
                save_dir='res',
                nb_hist=1024,
                kernel_size=32,
                nb_filters=32,
                ignore_boundaries=True,
                verbose=2,
                nb_conv=4,
                learning_rate=0.0005,
                use_separable=[True, True, False, False],
                nb_epoch=1000)

Adjust the name to point to the results:

In [ ]:
res_name = '/content/res/20210925_132436'
res = flammkuchen.load(f'{res_name}_results.h5')

Inspect the history of the training and validation loss

In [ ]:
plt.plot(res['fit_hist']['loss'], label='train')
plt.plot(res['fit_hist']['val_loss'], label='val')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

Plot the test results:

In [ ]:
# t0, t1 = 1_020_000, 1_040_000  # dmel tutorial dataset
t0, t1 = 40_000, 60_000

plt.figure(figsize=(40, 8))
plt.subplot(311)
plt.plot(res['x_test'][t0:t1], 'k')
plt.ylabel('Audio')

plt.subplot(312)
plt.plot(res['y_test'][t0:t1, 1:])
plt.ylabel('Prediction targets')

plt.subplot(313)
plt.plot(res['y_pred'][t0:t1, 1:])
plt.ylabel('Predictions')

plt.show()

You can download the model results via the file tab on the left, from `/contest/res`

## Predict on new data
Load a new recording for prediction

In [ ]:
model, params = das.utils.load_model_and_params(res_name)  # load the model and runtime parameters
ds = das.npy_dir.load('/content/drive/MyDrive/Dmoj.wrigleyi.npy', memmap_dirs=['train','val'])  # load the new data
print(ds)
x = ds['test']['x']

Run inference - this will calculate the confidence score and extract segment boundaries and event times.

In [ ]:
events, segments, class_probabilities, _ = das.predict.predict(x, model=model, params=params, verbose=1)

In [ ]:
t0, t1 = 42_000, 48_000
plt.figure(figsize=(20, 8))
plt.plot(x[t0:t1, 0], alpha=0.25, c='k', label='Audio')
plt.plot(class_probabilities[t0:t1, 1:], label='Prediction')
plt.xlim(0, t1-t0)
plt.legend(['Audio', 'Prediction'])
plt.show()